- OLMOE
- Llama3.2-Vision
- Phi3.5-Vision

In [1]:
import requests
import torch
from PIL import Image

from frames.representations import FrameUnembeddingRepresentation
from frames.models import VisionLanguageHuggingFaceModel
from frames.nlp.synsets import SupportedLanguages

In [2]:
fur = FrameUnembeddingRepresentation.from_model_id(
    id="meta-llama/Llama-3.2-11B-Vision-Instruct",
    model_cls=VisionLanguageHuggingFaceModel,
    language_codes=SupportedLanguages.Llama,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

2024-12-11 17:53:42.163 | INFO     | frames.models.hf.base:__init__:88 - Loaded model: meta-llama/Llama-3.2-11B-Vision-Instruct
2024-12-11 17:53:42.165 | WARNING  | frames.models.hf.base:__init__:89 - memory cost: 20351 Mb


In [3]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/86/Woman_at_Lover%27s_Bridge_Tanjung_Sepat_%28cropped%29.jpg/1024px-Woman_at_Lover%27s_Bridge_Tanjung_Sepat_%28cropped%29.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image.size

(1024, 1278)

In [ ]:
inputs = [
    {"image": image, "text": ["What can you say about it?"]},
]

out = fur.model.generate(
    **inputs[0],
    max_new_tokens=64,
    do_sample=False,
    temperature=None,
    top_p=1,
)

print(fur.model.processor.batch_decode(out)[0])

NameError: name 'text' is not defined

In [ ]:
text = "What can you say about it?"
text_input = fur.model._make_text_input(text)
data = ([image] * 3, [text_input] * 3)
fur.model.processor(*data)


{'input_ids': [[128000, 128000, 128006, 882, 128007, 271, 128256, 3923, 649, 499, 2019, 922, 433, 30, 128009, 128006, 78191, 128007, 271], [128000, 128000, 128006, 882, 128007, 271, 128256, 3923, 649, 499, 2019, 922, 433, 30, 128009, 128006, 78191, 128007, 271], [128000, 128000, 128006, 882, 128007, 271, 128256, 3923, 649, 499, 2019, 922, 433, 30, 128009, 128006, 78191, 128007, 271]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'pixel_values': array([[[[[[ 1.6383677 ,  1.6383677 ,  1.6383677 , ...,  1.6383677 ,
             1.6383677 ,  1.6383677 ],
           [ 1.6383677 ,  1.6383677 ,  1.6383677 , ...,  1.6383677 ,
             1.6383677 ,  1.6383677 ],
           [ 1.6383677 ,  1.6383677 ,  1.6383677 , ...,  1.6383677 ,
             1.6383677 ,  1.6383677 ],
           ...,
           [ 1.0544306 ,  1.0690291 ,  1.0836275 , ..., -1.

In [ ]:
fur.quick_generate_with_topk_guide(
    inputs,
    guide=["woman.n.01", "male.n.01"],
    min_lemmas_per_synset=3,
    max_token_count=3,
    k=2,
    steps=16,
)[0]

  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: VisionLanguageHuggingFaceModel.make_input() missing 1 required positional argument: 'text'